### 数据收集


In [1]:
import pandas as pd
import pymysql
db = pymysql.connect('mysql-backup.dian.so','yihui','GMYAvsgzE9fRRC1r','oss_view') # 打开数据库连接
sql='''
select e.id as '用户id', nick_name as '花名',
e.sex as '性别',
education_first as '学历',
r.birth_date as '生日',
e.level_id as '级别' 
,recruite_channel as '来源' ,
e.department_id  as '部门id-1',
(
select name from agent_department ad where id=e.department_id
)as '部门名称-1',
(
select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=e.department_id 
)
)as '部门名称-2',
(
select id from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=e.department_id 
)
)as '部门id-2',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=e.department_id 
  )
 )
)as '部门名称-3',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=e.department_id 
    )
  )
 )
)as '部门名称-4',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=e.department_id 

          )
    )
  )
 )
)as '部门名称-5',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=e.department_id 
                      )

          )
    )
  )
 )
)as '部门名称-6',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                                            select parent_id from agent_department ad4 where id=e.department_id 

                      )
                      )

          )
    )
  )
 )
)as '部门名称-7'

from  hr_resume as r left JOIN  agent_employee as e
on e.id=r.employee_id
where  real_status=1 and status=0 and agent_id=1
order by e.id desc
'''

user=pd.read_sql(sql=sql,con=db)
db.close()

In [2]:
user.head(10)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7
0,24277,时宝,1,大专,1995-08-18,2,None,1147,台州BDM_彦之,台州,1127,浙江分区,华东区,直营事业部,商务发展部,杭州小电科技股份有限公司
1,24276,阿舟,1,大专,1994-04-07,2,None,2306,黔东南小分队虚拟BDM,贵州外区,3029,云贵分区,西南区,直营事业部,商务发展部,杭州小电科技股份有限公司
2,24274,迪菲,2,大专,1993-10-08,2,None,1474,南宁BDM_柚柿,南宁,1038,华南区,直营事业部,商务发展部,杭州小电科技股份有限公司,None
3,24273,星灵,1,本科,1991-09-06,2,None,2043,杭州三区虚拟BDM,杭州三区,2571,杭州分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司
4,24272,艾悦,1,高中,2002-07-19,2,None,2045,福州BDM_瞿麦,福州,249,东南区,直营事业部,商务发展部,杭州小电科技股份有限公司,None
5,24271,靓凯,1,高中,1998-12-01,2,None,2328,拉萨BDM_索屠,拉萨,2327,川藏分区,西南区,直营事业部,商务发展部,杭州小电科技股份有限公司
6,24270,水秀,2,大专,1991-07-09,2,None,3240,泰安,华北大区,1978,仓库管理部,仓储部,供应链中心,杭州小电科技股份有限公司,None
7,24269,薄云,1,中专,1990-05-10,2,None,2294,焦作小分队BDM_古堡,河南外区,2976,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None
8,24268,春竹,1,大专,1995-10-02,2,None,2918,杭州BDM_深海,杭州二区,2097,杭州分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司
9,24266,夜雀,1,本科,1995-10-10,2,None,1642,淄博小分队BDM_万象,山东外区,2718,山东分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司


### 数据处理

In [7]:
"""
将性别标记位转换成文字
"""
def get_sex(sex):
    if sex==1:
        return '男'
    else:
        return '女'

In [8]:
user['性别']=user.apply(lambda x: get_sex(x['性别']),axis=1)


In [13]:
def get_birth_day(birth_date):
    dt95='1995-01-01'
    dt90='1990-01-01'
    dt85='1985-01-01'
    dt80='1980-01-01'
    birth=str(birth_date)
    if birth>dt95:
        return '95后'
    if birth<dt95 and birth>dt90:
        return '90-95'
    if birth>dt85 and birth<dt90:
        return '85-90'
    if birth>dt80 and birth<dt85:
        return '80-85'
    if birth<dt80:
        return '70后'
    return '未知'

In [14]:
user['年龄段']=user.apply(lambda x: get_birth_day(x['生日']),axis=1)


In [15]:
user.head(10)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7,年龄段
0,24277,时宝,男,大专,1995-08-18,2,None,1147,台州BDM_彦之,台州,1127,浙江分区,华东区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
1,24276,阿舟,男,大专,1994-04-07,2,None,2306,黔东南小分队虚拟BDM,贵州外区,3029,云贵分区,西南区,直营事业部,商务发展部,杭州小电科技股份有限公司,90-95
2,24274,迪菲,女,大专,1993-10-08,2,None,1474,南宁BDM_柚柿,南宁,1038,华南区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,90-95
3,24273,星灵,男,本科,1991-09-06,2,None,2043,杭州三区虚拟BDM,杭州三区,2571,杭州分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,90-95
4,24272,艾悦,男,高中,2002-07-19,2,None,2045,福州BDM_瞿麦,福州,249,东南区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后
5,24271,靓凯,男,高中,1998-12-01,2,None,2328,拉萨BDM_索屠,拉萨,2327,川藏分区,西南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
6,24270,水秀,女,大专,1991-07-09,2,None,3240,泰安,华北大区,1978,仓库管理部,仓储部,供应链中心,杭州小电科技股份有限公司,None,90-95
7,24269,薄云,男,中专,1990-05-10,2,None,2294,焦作小分队BDM_古堡,河南外区,2976,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,90-95
8,24268,春竹,男,大专,1995-10-02,2,None,2918,杭州BDM_深海,杭州二区,2097,杭州分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
9,24266,夜雀,男,本科,1995-10-10,2,None,1642,淄博小分队BDM_万象,山东外区,2718,山东分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后


### 数据探索


In [17]:
### 小电性别分布
sex_pie=user.groupby('性别').size()
sex_pie

性别
女     925
男    4207
dtype: int64

In [18]:
### 商务发展部性别分布
biz_sex_pie=user[ (user['部门名称-2']=='商务发展部') | (user['部门名称-3']=='商务发展部')| (user['部门名称-4']=='商务发展部')| (user['部门名称-5']=='商务发展部')| (user['部门名称-6']=='商务发展部')| (user['部门名称-7']=='商务发展部')].groupby('性别').size()
biz_sex_pie

性别
女     481
男    3571
dtype: int64

In [19]:
### 产品研发中心性别分布
doy_sex_pie=user[ (user['部门名称-2']=='产品研发中心') | (user['部门名称-3']=='产品研发中心')| (user['部门名称-4']=='产品研发中心')| (user['部门名称-5']=='产品研发中心')| (user['部门名称-6']=='产品研发中心')| (user['部门名称-7']=='产品研发中心')].groupby('性别').size()
doy_sex_pie

性别
女     37
男    191
dtype: int64

In [20]:
### 产品研发中心学历分布
doy_pie=user[ (user['部门名称-2']=='产品研发中心') | (user['部门名称-3']=='产品研发中心')| (user['部门名称-4']=='产品研发中心')| (user['部门名称-5']=='产品研发中心')| (user['部门名称-6']=='产品研发中心')| (user['部门名称-7']=='产品研发中心')].groupby('学历').size()
doy_pie

学历
中专         1
大专        31
本科       172
硕士研究生     23
高中         1
dtype: int64

In [22]:
### 商务发展部学历分布
biz_pie=user[ (user['部门名称-2']=='商务发展部') | (user['部门名称-3']=='商务发展部')| (user['部门名称-4']=='商务发展部')| (user['部门名称-5']=='商务发展部')| (user['部门名称-6']=='商务发展部')| (user['部门名称-7']=='商务发展部')].groupby('学历').size()
biz_pie

学历
中专        494
初中         31
大专       2385
小学          1
本科        663
硕士研究生       2
高中        476
dtype: int64

In [23]:
### 小电学历分布
all_pie=user.groupby('学历').size()
all_pie

学历
MBA         4
中专        550
初中         37
大专       2767
小学          1
本科       1184
硕士研究生      46
高中        543
dtype: int64

In [24]:
### 小电年龄段分布
age_pie=user.groupby('年龄段').size()
age_pie

年龄段
70后        18
80-85     147
85-90     865
90-95    2372
95后      1730
dtype: int64

### 数据展现


In [25]:
### 饼状图通用方法
def user_pie(title_str,lables_str,values_str):
    import plotly.offline as py
    from plotly.graph_objs import Scatter, Layout
    import plotly.graph_objs as go

    py.init_notebook_mode(connected=True)

    pyplt = py.offline.plot
    labels = lables_str
    values = values_str
    trace = [go.Pie(labels=labels, values=values)]
    layout = go.Layout(
        title = title_str,
    )
    figure = go.Figure(data = trace, layout = layout)
    py.iplot(figure)

In [26]:
user_pie('小电整体学历分布比例',all_pie.index.tolist(),all_pie.values.tolist())


In [27]:
user_pie('小电商务发展部学历分布比例',biz_pie.index.tolist(),biz_pie.values.tolist())


In [28]:
user_pie('小电产品研发部学历分布比例',doy_pie.index.tolist(),doy_pie.values.tolist())


In [29]:
user_pie('小电产品研发部性别分布比例',doy_sex_pie.index.tolist(),doy_sex_pie.values.tolist())


In [30]:
user_pie('小电商务发展部性别分布比例',biz_sex_pie.index.tolist(),biz_sex_pie.values.tolist())


In [31]:
user_pie('小电年龄段分布',age_pie.index.tolist(),age_pie.values.tolist())


### 报告撰写
学历
小电整体学历大专是主流
但是部门之间差异比较大
在商务发展部，大专是主流
在产品研发部，本科和硕士是主流
性别
小电是个男多女少的公司，想找妹子的需要避开
年龄
小电整体年龄结构比较年轻，90后是绝对主流
